In [1]:
%env XDG_CACHE=/workspace/.cache
%env HF_HOME=/workspace/.cache/huggingface


env: XDG_CACHE=/workspace/.cache
env: HF_HOME=/workspace/.cache/huggingface


In [2]:
from datasets import load_dataset
import pandas as pd
import random


In [7]:
# Load dataset
dataset = load_dataset("projecte-aina/teca")
df = dataset['train'].to_pandas()
# Parameters
seed = 55
few_shot_count = 3
random.seed(seed)

entries = {
    "prompt": [],
    "answer": [],
    "numeric_label": []
}
labels = {
    1: "neutre",
    0: "implicació",
    2: "contradicció"
}


while len(df) > few_shot_count + 1:
    try:
        examples = []
        examples.append(df.query("label == 0").sample(1, random_state=seed))
        df = df.drop(index=examples[-1].index)
        examples.append(df.query("label == 1").sample(1, random_state=seed))
        df = df.drop(index=examples[-1].index)
        examples.append(df.query("label == 2").sample(1, random_state=seed))
        df = df.drop(index=examples[-1].index)
        examples.append(df.sample(few_shot_count + 1 - 3, random_state=seed))
        df = df.drop(index=examples[-1].index)
    except:
        break

    samples = pd.concat(examples)

    prompt = ""
    for elm in samples.iloc[:-1].itertuples():
        prompt += f"premissa: {elm.premise}\nhipòtesi: {elm.hypothesis}\netiqueta: {labels[elm.label]}"
        prompt += "\n----\n"

    last_element = samples.iloc[-1]
    prompt += f"premissa: {last_element.premise}\nhipòtesi: {last_element.hypothesis}\netiqueta:"
    entries["prompt"].append(prompt)
    entries["answer"].append(labels[last_element.label])
    entries["numeric_label"].append(last_element.label)


teca = pd.DataFrame(entries)
teca.to_csv("teca.csv", index=False)


In [8]:
teca = pd.read_csv("teca.csv")
print(teca.iloc[0]['prompt'])


premissa: Nosaltres, per exemple, no estem d'acord de ser la crossa del PSOE.
hipòtesi: No volem que el PSOE es recolzi en nosaltres.
etiqueta: implicació
----
premissa: Per una banda es cert que aqui no es reconeix el talent nacional i (entre l'alta burgesia del Liceu i Perelada encara menys).
hipòtesi: Tenim veritables talents en l'àmbit musical, especialment els darrers 20 anys.
etiqueta: neutre
----
premissa: Lluís Llach recrimina el paper dels comuns en la votació al parlament del 27 d’octubre
hipòtesi: Lluís Llach aplaudeix l'acció dels comuns
etiqueta: contradicció
----
premissa: L'ambientació de la novel·la es extraordinària+B951 els paisatges i els personatges són absolutament vius.
hipòtesi: Les descripcions de la novel·la són tan vívides que sembla que els paisatges i personatges siguin reals.
etiqueta: implicació
----
premissa: Els llops, bisons, cignes i els falcons també són presents.
hipòtesi: Hi ha mamífers i aus.
etiqueta: implicació
----
premissa: La culpa de tot la t

In [6]:
teca


,prompt,answer,numeric_label
0,"premissa: Nosaltres, per exemple, no estem d'a...",implicació,0
1,premissa: Els CDR convoquen una protesta davan...,contradicció,2
2,premissa: Vaig incorporar me a la feina quan e...,contradicció,2
3,premissa: La tradició i la modernitat conviuen...,neutre,1
4,premissa: Senda ofereix pràctiques d'un períod...,implicació,0
...,...,...,...
2815,"premissa: Així, per exemple, ""do-re-mi"" es pod...",contradicció,2
2816,premissa: La resposta de l'estat i les adminis...,contradicció,2
2817,premissa: També volen disfrutar de la neu d'un...,neutre,1
2818,"premissa: Sí, a mi també em va atacar un vampi...",contradicció,2
